In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimPF6_PR import configuration

# Load data from csv
data = pd.read_csv('HFCSIL_g.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    #if data contains R_125
    
    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(100000)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
    m.fs.state_block.mole_frac_comp["bmimPF6"].fix(0.96)
    
    m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
    m.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].fix(-0.026)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].fix(-0.07995)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].fix(0.91)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].fix(0.05)
    
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
    m.fs.state_block.mole_frac_comp["bmimPF6"].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimPF6'].fix(float(data["x_IL"])) 
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'R32'].setub(3)

    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R32', 'bmimPF6'].setub(3)

    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R32'].setub(3)
    
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa_A['bmimPF6', 'R125'].setub(3)

    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setlb(-3)
    m.fs.properties.PR_kappa_A['R125', 'bmimPF6'].setub(3)
    
    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.7312, "x_IL":0.7312,  
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-02-09 11:27:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 1


In [7]:
variable_name = [
#                 "fs.properties.PR_kappa_A['R32', 'R125']",
#                 "fs.properties.PR_kappa_A['R125', 'R32']",
                "fs.properties.PR_kappa_A['R32', 'bmimPF6']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R32']",
                "fs.properties.PR_kappa_A['bmimPF6', 'R125']",
                "fs.properties.PR_kappa_A['R125', 'bmimPF6']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-9

In [9]:
solver_options = {'tol': 1e-7}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2022-02-09 11:27:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-09 11:27:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-09 11:27:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-09 11:27:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-09 11:27:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-02-09 11:27:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more inf

  63  9.4729969e-06 7.85e-03 1.73e+00  -5.7 2.65e-01  -2.6 9.87e-01 4.70e-03h  4
  64  9.4730006e-06 7.81e-03 1.66e+01  -5.7 3.17e-01  -3.0 1.00e+00 5.49e-03h  4
  65  9.4731707e-06 7.74e-03 1.89e+02  -5.7 3.49e+00  -3.5 1.00e+00 9.55e-03h  3
  66  9.4839185e-06 7.23e-03 2.07e+02  -5.7 1.80e+01  -4.0 1.25e-01 6.60e-02h  2
  67  9.4839008e-06 7.23e-03 4.41e+01  -5.7 9.28e+00  -2.7 1.28e-02 2.03e-04h  6
  68  9.4838998e-06 7.23e-03 4.40e+01  -5.7 3.24e+00  -1.3 1.49e-02 5.36e-04h  6
  69  9.4838998e-06 7.11e-03 6.28e+01  -5.7 1.41e-01  -0.0 1.00e+00 1.70e-02h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  70  9.4838997e-06 6.97e-03 1.41e+02  -5.7 1.32e-01  -0.5 4.85e-01 1.88e-02h  3
  71  9.4839006e-06 6.63e-03 4.02e+02  -5.7 1.06e-01  -0.1 1.00e+00 4.92e-02h  2
  72  9.4839025e-06 6.14e-03 7.21e+02  -5.7 2.78e-01  -0.5 1.00e+00 7.31e-02h  2
  73  9.4839010e-06 5.85e-03 5.90e+02  -5.7 6.37e-01  -0.1 9.10e-02 4.84e-02w  1
  74  9.4839092e-06 2.82e-03

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.000000

The values for the parameters are as follows:
fs.properties.PR_kappa_A[R32,bmimPF6] = -0.043507639771678236
fs.properties.PR_kappa_A[bmimPF6,R32] = -0.35057012196511156
fs.properties.PR_kappa_A[bmimPF6,R125] = 1.67786679559769
fs.properties.PR_kappa_A[R125,bmimPF6] = 0.06043553169992497


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])


In [12]:
from idaes.core.util.model_statistics import report_statistics
print(report_statistics(pest.ef_instance))


Model Statistics  -  _Q_opt 

Degrees of Freedom: 10 

Total No. Variables: 486 
    No. Fixed Variables: 366
    No. Unused Variables: 144 (Fixed):144)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 110 
    No. Equality Constraints: 110 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 7 (Deactivated: 6)

No. Blocks: 57 (Deactivated: 0) 
No. Expressions: 270 

None
